# Project 1
Konstantinos Konstantinidis 2546

Nikolaos Stavrinos 2631

In [1]:
import pandas as pd

result = pd.read_excel("sales.xlsx")

result.head()

,ID,Prod,Quant,Val,Insp
0,v1,p1,182.0,1665.0,unkn
1,v2,p1,3072.0,8780.0,unkn
2,v3,p1,20393.0,76990.0,unkn
3,v4,p1,112.0,1100.0,unkn
4,v3,p1,6164.0,20260.0,unkn


## Finding the Unit Price of each product

In [2]:
result["Unit_price"]=result[["Val"]].div(result.Quant, axis=0)
print(result.size)

2406876


## We will use the median unit price to represent the typical price at which a product is sold

In [3]:
uprice_by_prod=pd.DataFrame()
uprice_by_prod["Median"] = result["Unit_price"].groupby(result["Prod"], group_keys = True).agg(Median='median').dropna()['Median']

print("Cheap")
print(uprice_by_prod.sort_values("Median", ascending=True).head(5))
print("\nExpensive")
print(uprice_by_prod.sort_values("Median",  ascending=False).head(5))



Cheap
         Median
Prod           
p560   0.016885
p559   0.018844
p4195  0.030259
p601   0.055223
p563   0.055764

Expensive
            Median
Prod              
p3689  9204.195372
p2453   456.078431
p2452   329.313725
p2456   304.851485
p2459   283.811881


## Unknown Values

In [4]:
result[result['Quant'].isna() & result['Val'].isna()]

,ID,Prod,Quant,Val,Insp,Unit_price
3227,v29,p808,NaN,NaN,unkn,NaN
4166,v453,p921,NaN,NaN,unkn,NaN
5102,v431,p1035,NaN,NaN,unkn,NaN
5211,v431,p1,NaN,NaN,unkn,NaN
5213,v431,p1,NaN,NaN,unkn,NaN
...,...,...,...,...,...,...
396552,v4834,p3801,NaN,NaN,unkn,NaN
396751,v2719,p3825,NaN,NaN,unkn,NaN
400010,v1247,p4088,NaN,NaN,unkn,NaN
400094,v1085,p4089,NaN,NaN,unkn,NaN


## Removing those cells with both Quant and Val as Nan

In [5]:
result=result[~(result['Quant'].isna() & result['Val'].isna())]

## There are two products (p2442 and p2443) that have all their transactions with unknown values of the quantity

In [6]:
result=result[~(result['Prod']=="p2442" )]
result=result[~(result['Prod']=="p2443" )]

## Remaining Products

In [7]:
len(result.groupby(result["Prod"], group_keys = True))

4546

## Calculate any of the two possibly missing values (Quant and Val).

In [8]:
import numpy as np

No_Fraud=result[result["Insp"].values!="fraud"]
tPrice= No_Fraud["Unit_price"].groupby(No_Fraud["Prod"], group_keys = True).agg(Median='median').dropna()['Median']



result["Quant"]= np.where(pd.isna(result["Quant"]), np.ceil(result['Val'].values/(tPrice.loc[result['Prod']].values)), result["Quant"])


In [9]:
print(result["Quant"].isna().sum())

0


In [10]:

result["Val"]= np.where(pd.isna(result["Val"]), np.ceil(result['Quant'].values*(tPrice.loc[result['Prod']].values)), result["Val"])


In [11]:
print(result["Val"].isna().sum())

0


## Recalculating Unit Price

In [12]:
result["Unit_price"]=result[["Val"]].div(result.Quant, axis=0)

## Removing the outliers with the use of the IQR method

In [13]:
print(result.size)
Q1 = result.quantile(0.25)
Q3 = result.quantile(0.75)
IQR = Q3 - Q1

result = result[~((result < (Q1 - 1.5 * IQR)) |(result > (Q3 + 1.5 * IQR))).any(axis=1)]
result.size

2401224


<ipython-input-13-a8a248d3126b>:6: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  result = result[~((result < (Q1 - 1.5 * IQR)) |(result > (Q3 + 1.5 * IQR))).any(axis=1)]
<ipython-input-13-a8a248d3126b>:6: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  result = result[~((result < (Q1 - 1.5 * IQR)) |(result > (Q3 + 1.5 * IQR))).any(axis=1)]


1741110

## Semi Supervised learning Model

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import svm




Labeled_data=result[result["Insp"].values!="unkn"]

Labeled_data

,ID,Prod,Quant,Val,Insp,Insp_code,Prod_code,Unit_price
51,v45,p11,260.0,1925.0,ok,1.0,112.0,7.403846
77,v46,p12,475.0,2600.0,ok,1.0,222.0,5.473684
79,v48,p12,433.0,3395.0,ok,1.0,222.0,7.840647
91,v49,p14,212.0,1545.0,ok,1.0,432.0,7.287736
380,v68,p59,107.0,1015.0,ok,1.0,4007.0,9.485981
...,...,...,...,...,...,...,...,...
400767,v3853,p4095,732.0,15110.0,ok,1.0,3387.0,20.642077
400768,v5904,p4095,201.0,13640.0,ok,1.0,3387.0,67.860697
400789,v3848,p4096,123.0,7560.0,fraud,0.0,3388.0,61.463415
400797,v3179,p4096,907.0,17350.0,ok,1.0,3388.0,19.128997


In [7]:
X_train, X_test, y_train, y_test = train_test_split( Labeled_data[["Prod_code","Quant","Val"]],
                                                    Labeled_data[["Insp_code"]], test_size=0.30, random_state=42)

# all parameters not specified are set to their defaults
print(y_train.shape)
print(X_train.shape)

print(y_train.shape)


(5560, 1)
(5560, 3)
(5560, 1)


In [8]:
clf = svm.SVC(kernel='linear', probability=True,C=1).fit(X_train, y_train.values.ravel())

In [9]:
clf.score(X_test, y_test)


0.9446308724832215

In [10]:
Unlabeled_data=result[result["Insp"].values=="unkn"]
X_unlabeled=Unlabeled_data[["Prod_code","Quant","Val"]]

clp= clf.predict_proba(X_unlabeled)
pseudo_label=clf.predict(X_unlabeled)

#Unlabeled_data = pd.DataFrame(clp, columns = ['C1Prob', 'C2Prob']) 
Unlabeled_data['C1Prob'] = clp.T[0]
Unlabeled_data['C2Prob'] = clp.T[1] 
Unlabeled_data['Insp_code']=pseudo_label
Unlabeled_data['max']=Unlabeled_data[["C1Prob", "C2Prob"]].max(axis=1)
Unlabeled_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,ID,Prod,Quant,Val,Insp,Insp_code,Prod_code,Unit_price,C1Prob,C2Prob,max
0,v1,p1,182.0,1665.0,unkn,1.0,0.0,9.148352,0.051447,0.948553,0.948553
3,v4,p1,112.0,1100.0,unkn,1.0,0.0,9.821429,0.050338,0.949662,0.949662
5,v5,p2,104.0,1155.0,unkn,1.0,1095.0,11.105769,0.051208,0.948792,0.948792
6,v6,p2,350.0,5680.0,unkn,1.0,1095.0,16.228571,0.062153,0.937847,0.937847
7,v7,p2,200.0,4010.0,unkn,1.0,1095.0,20.050000,0.058101,0.941899,0.941899
...,...,...,...,...,...,...,...,...,...,...,...
401141,v5918,p4126,113.0,9330.0,unkn,1.0,3422.0,82.566372,0.076769,0.923231,0.923231
401142,v2540,p4126,118.0,3355.0,unkn,1.0,3422.0,28.432203,0.058370,0.941630,0.941630
401143,v5896,p4127,143.0,1835.0,unkn,1.0,3423.0,12.832168,0.054314,0.945686,0.945686
401144,v1796,p4127,102.0,1445.0,unkn,1.0,3423.0,14.166667,0.053481,0.946519,0.946519


In [11]:
frames = [Unlabeled_data[["Prod_code","Quant","Val","Insp_code"]], Labeled_data[["Prod_code","Quant","Val","Insp_code"]]]

final_data = pd.concat(frames)
final_data

,Prod_code,Quant,Val,Insp_code
0,0.0,182.0,1665.0,1.0
3,0.0,112.0,1100.0,1.0
5,1095.0,104.0,1155.0,1.0
6,1095.0,350.0,5680.0,1.0
7,1095.0,200.0,4010.0,1.0
...,...,...,...,...
400767,3387.0,732.0,15110.0,1.0
400768,3387.0,201.0,13640.0,1.0
400789,3388.0,123.0,7560.0,0.0
400797,3388.0,907.0,17350.0,1.0


In [12]:
X_train, X_test, y_train, y_test = train_test_split( final_data[["Prod_code","Quant","Val"]],
                                                    final_data[["Insp_code"]], test_size=0.30, random_state=42)

clf = svm.SVC(kernel='linear', probability=True,C=1).fit(X_train, y_train.values.ravel())

clf.score(X_test, y_test)


0.9984484943839868